<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20220926.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* customer analysis
* iterator

In [1]:
import pandas as pd 
import numpy as np
import plotnine as gg
from plotnine import ggplot, aes 
import matplotlib.pyplot as plt 
from datetime import datetime, date, time, timezone, timedelta 
import os
import re
from functools import reduce
from collections import defaultdict, Counter, ChainMap


# customer analysis

In [3]:
os.listdir()

['.config',
 'ecommerce-data.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 'KPMG_VI_New_raw_data_update_final.csv',
 'online_retail_II.xlsx',
 'scanner_data.csv',
 'test.csv',
 'sample_data']

In [8]:
df = pd.read_csv('KPMG_VI_New_raw_data_update_final.csv')
df = df[df.order_status == 'Approved']
df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,25-02-2017,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,21-05-2017,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,16-10-2017,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,31-08-2017,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,01-10-2017,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [9]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df['date'] = df['transaction_date'].to_numpy().astype('datetime64[D]')
df['week'] = df['date'] - df['date'].dt.weekday.astype('timedelta64[D]')
df['month'] = df['transaction_date'].to_numpy().astype('datetime64[M]')
df = df.rename(columns = {'customer_id':'id', 'list_price':'rev'}).groupby(['id','date','week','month'])['rev'].sum().reset_index()
df = df.sort_values(['id', 'date'], ascending = [1,0])
df.head()

,id,date,week,month,rev
10,1,2017-12-23,2017-12-18,2017-12-01,235.63
9,1,2017-12-14,2017-12-11,2017-12-01,230.91
8,1,2017-11-05,2017-10-30,2017-11-01,1720.70
7,1,2017-09-12,2017-09-11,2017-09-01,742.54
6,1,2017-06-04,2017-05-29,2017-06-01,1577.53


In [18]:
info_m = df.groupby(['id','month'])['rev'].sum().reset_index().rename(columns = {'rev':'money'})
info_m = pd.pivot_table(info_m, index = 'id', columns = 'month', values = 'money').reset_index()
info_m = pd.melt(info_m, id_vars = 'id', var_name = 'month', value_name = 'money').sort_values(['id','month'], ascending = [1,0])
info_m = info_m.groupby(['id']).head(3)
info_m = info_m.groupby(['id'])['money'].sum().reset_index()
info_m.head()

,id,money
0,1,2187.24
1,2,752.64
2,3,0.00
3,4,0.00
4,5,1812.75


In [19]:
info_f = df.groupby(['id','month'])['rev'].count().reset_index().rename(columns = {'rev':'freq'})
info_f = pd.pivot_table(info_f, index = 'id', columns = 'month', values = 'freq').reset_index()
info_f = pd.melt(info_f, id_vars = 'id', var_name = 'month', value_name = 'freq').sort_values(['id','freq'], ascending = [1,0])
info_f = info_f.groupby(['id']).head(3)
info_f = info_f.groupby(['id'])['freq'].sum().reset_index()
info_f.head()

,id,freq
0,1,6.0
1,2,3.0
2,3,5.0
3,4,2.0
4,5,4.0


In [22]:
date_max = df['date'].max() + pd.offsets.MonthEnd(1)
date_max

Timestamp('2017-12-31 00:00:00')

In [30]:
info_r = df.groupby(['id']).agg({'date' : lambda x : (date_max - x.max())}).reset_index().rename(columns = {'date':'recency'})
info_r['recency'] = info_r['recency'].dt.days
info_r.head()

,id,recency
0,1,8
1,2,55
2,3,103
3,4,196
4,5,17


In [33]:
info = pd.merge(pd.merge(info_r, info_f, 'left', 'id'), info_m, 'left', 'id')
info = info.assign(
    r = pd.qcut(info['recency'], 5, list('12345')),
    f = pd.qcut(info['freq'].rank(method = 'first'), 5, list('54321')),
    m = pd.qcut(info['money'].rank(method = 'first'), 5, list('54321'))
)
info['fm'] = info.r.astype(str) + info.m.astype(str)
info.head()

,id,recency,freq,money,r,f,m,fm
0,1,8,6.0,2187.24,1,1,2,12
1,2,55,3.0,752.64,3,5,4,34
2,3,103,5.0,0.00,5,2,5,55
3,4,196,2.0,0.00,5,5,5,55
4,5,17,4.0,1812.75,2,4,2,22


# iterator

In [35]:
mat = [list('ABC'), list('DEF'), list('GHI')]
a = {letters : [r, c] for r, rows in enumerate(mat) for c, letters in enumerate(rows)}
a

{'A': [0, 0],
 'B': [0, 1],
 'C': [0, 2],
 'D': [1, 0],
 'E': [1, 1],
 'F': [1, 2],
 'G': [2, 0],
 'H': [2, 1],
 'I': [2, 2]}

In [36]:
arr = np.empty([3,3], dtype = 'object')
for k, v in a.items() :
  arr[v[0],v[1]] = k
arr

array([['A', 'B', 'C'],
       ['D', 'E', 'F'],
       ['G', 'H', 'I']], dtype=object)